In [1]:
qwer = '''// post.entity.ts
export class Post {
  constructor(
    public id: string,
    public title: string,
    public content: string,
    public authorId: string
  ) {}
}

// post.repository.interface.ts
export interface IPostRepository {
  create(post: Post): Promise<Post>;
  findAll(): Promise<Post[]>;
  findById(id: string): Promise<Post | null>;
  update(id: string, post: Partial<Post>): Promise<Post | null>;
  delete(id: string): Promise<boolean>;
}

// post.repository.ts
import { Injectable } from '@nestjs/common';
import { Post } from './post.entity';
import { IPostRepository } from './post.repository.interface';

@Injectable()
export class PostRepository implements IPostRepository {
  private posts: Post[] = [];

  async create(post: Post): Promise<Post> {
    this.posts.push(post);
    return post;
  }

  async findAll(): Promise<Post[]> {
    return this.posts;
  }

  async findById(id: string): Promise<Post | null> {
    return this.posts.find(post => post.id === id) || null;
  }

  async update(id: string, postData: Partial<Post>): Promise<Post | null> {
    const post = await this.findById(id);
    if (!post) return null;
    Object.assign(post, postData);
    return post;
  }

  async delete(id: string): Promise<boolean> {
    const index = this.posts.findIndex(post => post.id === id);
    if (index === -1) return false;
    this.posts.splice(index, 1);
    return true;
  }
}

// post.service.ts
import { Injectable } from '@nestjs/common';
import { Post } from './post.entity';
import { IPostRepository } from './post.repository.interface';

@Injectable()
export class PostService {
  constructor(private postRepository: IPostRepository) {}

  async createPost(title: string, content: string, authorId: string): Promise<Post> {
    const id = Date.now().toString();
    const post = new Post(id, title, content, authorId);
    return this.postRepository.create(post);
  }

  async getAllPosts(): Promise<Post[]> {
    return this.postRepository.findAll();
  }

  async getPostById(id: string): Promise<Post | null> {
    return this.postRepository.findById(id);
  }

  async updatePost(id: string, postData: Partial<Post>): Promise<Post | null> {
    return this.postRepository.update(id, postData);
  }

  async deletePost(id: string): Promise<boolean> {
    return this.postRepository.delete(id);
  }
}

// post.controller.ts
import { Controller, Get, Post, Put, Delete, Body, Param } from '@nestjs/common';
import { PostService } from './post.service';
import { Post as PostEntity } from './post.entity';

@Controller('posts')
export class PostController {
  constructor(private postService: PostService) {}

  @Post()
  async createPost(@Body() postData: { title: string; content: string; authorId: string }): Promise<PostEntity> {
    return this.postService.createPost(postData.title, postData.content, postData.authorId);
  }

  @Get()
  async getAllPosts(): Promise<PostEntity[]> {
    return this.postService.getAllPosts();
  }

  @Get(':id')
  async getPostById(@Param('id') id: string): Promise<PostEntity | null> {
    return this.postService.getPostById(id);
  }

  @Put(':id')
  async updatePost(@Param('id') id: string, @Body() postData: Partial<PostEntity>): Promise<PostEntity | null> {
    return this.postService.updatePost(id, postData);
  }

  @Delete(':id')
  async deletePost(@Param('id') id: string): Promise<boolean> {
    return this.postService.deletePost(id);
  }
}

// post.module.ts
import { Module } from '@nestjs/common';
import { PostController } from './post.controller';
import { PostService } from './post.service';
import { PostRepository } from './post.repository';
import { IPostRepository } from './post.repository.interface';

@Module({
  controllers: [PostController],
  providers: [
    PostService,
    {
      provide: IPostRepository,
      useClass: PostRepository,
    },
  ],
})
export class PostModule {}'''

In [10]:
code_list = [i.strip() for i in qwer.split('//') if i]

In [11]:
code_list

['post.entity.ts\nexport class Post {\n  constructor(\n    public id: string,\n    public title: string,\n    public content: string,\n    public authorId: string\n  ) {}\n}',
 'post.repository.interface.ts\nexport interface IPostRepository {\n  create(post: Post): Promise<Post>;\n  findAll(): Promise<Post[]>;\n  findById(id: string): Promise<Post | null>;\n  update(id: string, post: Partial<Post>): Promise<Post | null>;\n  delete(id: string): Promise<boolean>;\n}',
 "post.repository.ts\nimport { Injectable } from '@nestjs/common';\nimport { Post } from './post.entity';\nimport { IPostRepository } from './post.repository.interface';\n\n@Injectable()\nexport class PostRepository implements IPostRepository {\n  private posts: Post[] = [];\n\n  async create(post: Post): Promise<Post> {\n    this.posts.push(post);\n    return post;\n  }\n\n  async findAll(): Promise<Post[]> {\n    return this.posts;\n  }\n\n  async findById(id: string): Promise<Post | null> {\n    return this.posts.find(po

In [19]:
from pathlib import Path 

src_dir = Path('src/post')
src_dir.mkdir(parents=True, exist_ok=True)

for code in code_list:
    _path = code.split('\n')[0]
    with open(src_dir / _path, 'wt', encoding='utf-8') as f:
        f.write("//" + code)